In [ ]:
import os
import psycopg2
import requests as req

from dotenv import load_dotenv
from psycopg2.extras import NamedTupleCursor

In [ ]:
def get_latest_nct_ids(count):
    db_username = os.getenv("AACT_USERNAME")
    db_password = os.getenv("AACT_PASSWORD")

    if not db_username or not db_password:
        print("Please make sure AACT_USERNAME and AACT_PASSWORD env variables are set in .env")
        return

    conn = psycopg2.connect(
        host="aact-db.ctti-clinicaltrials.org",
        database="aact",
        user=db_username,
        password=db_password,
        port="5432",
        cursor_factory=NamedTupleCursor
    )
    
    query= f"""
    SELECT
        s.nct_id
    FROM
        studies s
    LEFT JOIN sponsors sp ON s.nct_id = sp.nct_id
    LEFT JOIN outcome_analyses oa ON s.nct_id = oa.nct_id

    WHERE
        s.phase IN ('Phase 3')
        AND oa.p_value IS NOT NULL
        AND s.source = 'Pfizer'
    LIMIT {count};
    """
    
    with conn:
        with conn.cursor() as curs:
            curs.execute(query)
            nct_ids = [rec.nct_id for rec in curs.fetchall()]
    
    nct_ids = list(set(nct_ids))
    nct_ids.sort()

    return nct_ids

In [ ]:
list_of_nct_id = get_latest_nct_ids(100000)

In [ ]:
len(list_of_nct_id)

In [ ]:
list_of_nct_id